In [37]:
import sys
sys.path.append('../src')
import json
import pandas as pd
from dataframeFactory import get_data_frame
import statsmodels.api as sm
import numpy as np

# Load the JSON data from a file
with open('../data/raw_data/db.json') as f:
    json_data = json.load(f)

# run add_predictions_gauss_regr on the data to add column 'uncertainty'
filtered_data = get_data_frame(json_data)


# Initialize 'isMeasured' with a default value
filtered_data['isMeasured'] = None  

# Use boolean indexing to set values in 'isMeasured' column
filtered_data.loc[(filtered_data['all_measurements'] == 0), 'isMeasured'] = 1
filtered_data.loc[(filtered_data['all_measurements'] != 0), 'isMeasured'] = 0

# Convert 'isMeasured' to numeric (if it's not already)
filtered_data['isMeasured'] = pd.to_numeric(filtered_data['isMeasured'], errors='coerce')

# Add a constant term for the logistic regression
filtered_data['const'] = 1
X = filtered_data[['const', 'all_stability']]
y = filtered_data['isMeasured']

# Logistic regression model
logit_model = sm.Logit(y, X)

# Fit the model
result = logit_model.fit()

# Display the summary
print(result.summary())



Optimization terminated successfully.
         Current function value: 0.685459
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:             isMeasured   No. Observations:                23727
Model:                          Logit   Df Residuals:                    23725
Method:                           MLE   Df Model:                            1
Date:                Thu, 25 Jan 2024   Pseudo R-squ.:               1.084e-05
Time:                        18:49:07   Log-Likelihood:                -16264.
converged:                       True   LL-Null:                       -16264.
Covariance Type:            nonrobust   LLR p-value:                    0.5526
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.2354      0.026     -8.995      0.000      -0.287      -0.184
all_stability    -0.

In [29]:
with open('../data/raw_data/db.json') as f:
    json_data = json.load(f)

print(len(json_data['features']))

with open('../data/output_withStateId.json') as f:
    json_data = json.load(f)

print(len(json_data['features']))

with open('../data/output_rbf_40_60.json') as f:
    json_data = json.load(f)

print(len(json_data['features']))

23727
23727
23727


In [19]:
# Null hypothesis (H₀): There is no significant association between the presence of a measurement and stability greater than 0.5.
# Alternative hypothesis (H₁): There is a significant association between the presence of a measurement and stability greater than 0.5.


# Creating binary variables 'Measured' and 'High_Stability'.
filtered_data['Measured'] = (filtered_data['all_measurements'] > 0).astype(int)  # 1 for measurement, 0 for no measurement

filtered_data['High_Stability'] = (filtered_data['all_stability'] > 0.5).astype(int)  # 1 for high_stability > 0.5, 0 otherwise

# Adding a constant term for logistic regression.
filtered_data['const'] = 1

# Setting up independent variables 'X' (constant and 'Measured') and dependent variable 'y' ('High_Stability').
X = filtered_data[['const', 'Measured']]
y = filtered_data['High_Stability']

# Fitting logistic regression model using Statsmodels.
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Printing summary of logistic regression results.
print(result.summary())

# Concluding that the presence of a measurement ("Measured") is significantly associated with higher likelihood of high stability (>0.5).

Optimization terminated successfully.
         Current function value: 0.033435
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         High_Stability   No. Observations:                23727
Model:                          Logit   Df Residuals:                    23725
Method:                           MLE   Df Model:                            1
Date:                Thu, 25 Jan 2024   Pseudo R-squ.:                 0.02888
Time:                        18:32:44   Log-Likelihood:                -793.32
converged:                       True   LL-Null:                       -816.92
Covariance Type:            nonrobust   LLR p-value:                 6.434e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.2025      0.218     28.395      0.000       5.774       6.631
Measured      -1.4225      0